# **DATA VISUALISATION**  
Dans cette partie, nous allons voir ensemble comment nous pouvons mettre en valeur les données nettoyées avec de la visualisation.

Comme toujours, faire des visuels impactants nécessite de:
- Manipuler et transofrmer la données pour obtenir les valeurs que l'on souhaite
- Poser les bases du visuel (80% du travail - 20 % du temps)
- Travailler le visuel pour mettre en valeur le message (20% du travail - 80% du temps)

Nous utiliserons la module plotly.express (px) de la librairie plotly, très pratique pour sa syntaxe simple.  
Il permet de se familiariser avec l'environnement plotly, avant de pouvoir passer au module plotly.graph_objects (go),  
dont la syntaxe est plus ardue mais les possibilités bien supérieures.  
Voici le lien de l'API plotly pour voir les différents graphiques possibles : https://plotly.com/python-api-reference/  

Le graphique que nous faire ensemble dans ce Notebook est un visuel que nous avions vu ensemble lors de la séance 1  
et que Vertigo vend à ses clients chaque semaine : le profil par cible dans un graphique en barres empilées.

Ce que vous devez faire est indiqué dans les cellules de commentaires, il vous reste à écrire le code.  

### **Importer les librairies nécessaires :**  
- numpy
- pandas
- plotly.express (alias px)
- plotly.graph_objects (alias go)

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects as go

### **Importer les données nettoyées lors de la phase de Nettoyage**
Ne gardez que les colonnes suivantes : vague_cine_inv, pid, age, gender, s3ad, q2ad, sem_cine 
Nommer la variable data

In [2]:
data = (
    pd.read_pickle('data/clean_step_1.pkl')
    .loc[:, ['vague_cine_inv', 'pid', 'age', 'gender', 's3ad', 'q2ad', 'sem_cine']]
)

data.head()

,vague_cine_inv,pid,age,gender,s3ad,q2ad,sem_cine
0,2326,3162713,57,1,2,2921,2326
5,2326,5203821,50,1,2,2964,2326
6,2326,5361385,40,1,4,2966,2326
10,2326,9664953,65,1,4,2946,2326
12,2326,23992569,68,1,2,2964,2326


### **Importer le fichier des poids statistiques**
Nommer la variable weights  
Si vous avez des une erreur UnicodeDecodeError, il faut que vous utilisiez le paramètre suivant : encoding='latin'

In [3]:
weights = pd.read_csv('data/poids_statistiques.csv', sep="\t", encoding='latin-1')
weights.head()

,vague_cine_inv,pid,age,q2ad,Entrées Extrapolées,mois_cine
0,2326,1488240567,72,2964,4883.573695,2307
1,2326,290080841,49,2939,808.221050,2307
2,2326,1561725024,46,2964,1508.842008,2307
3,2326,1488239508,32,2964,1352.103424,2307
4,2326,266199476,60,2964,961.496080,2307


### **Réaliser un INNER JOIN entre data et weights**
Vous devez obtenir 9802 lignes  
Rappelez-vous des conditions d'unicité d'une ligne  
Nommer la variable data_weighted

In [5]:
data_weighted = data.merge(weights, how='inner', on=['vague_cine_inv', 'pid', 'age', 'q2ad'])
data_weighted

,vague_cine_inv,pid,age,gender,s3ad,q2ad,sem_cine,Entrées Extrapolées,mois_cine
0,2326,3162713,57,1,2,2921,2326,1452.762352,2307
1,2326,5203821,50,1,2,2964,2326,1199.758955,2307
2,2326,5361385,40,1,4,2966,2326,1885.009678,2307
3,2326,9664953,65,1,4,2946,2326,627.760083,2307
4,2326,23992569,68,1,2,2964,2326,3463.268595,2307
...,...,...,...,...,...,...,...,...,...
9797,2331,1976744286,8,2,4,2988,2330,1770.753674,2307
9798,2331,1988359483,36,2,2,2975,2330,1319.389671,2307
9799,2331,1988359483,36,2,2,2937,2330,1147.117041,2307
9800,2331,2002828674,28,2,5,2954,2330,4968.921297,2307


### **Créer une colonne permettant d'avoir les catégories d'âge suivantes :**
- 3-14 ans
- 15-24 ans
- 25-34 ans
- 35-49 ans
- 50-59 ans
- 60 ans et +

Nommer la colonne 'age_cat_6'  
Assurez-vous que la colonne soit bien de type category au moment où vous la créez

In [10]:
data_weighted['age_cat_6'] = pd.cut(data_weighted['age'], 
        bins=[0, 14, 24, 34, 49, 59, np.inf], 
        labels=['3-14 ans', '15-24 ans', '25-34 ans', '35-49 ans', '50-59 ans', '60 ans et +',]
    )
data_weighted['age_cat_6'].unique()

['50-59 ans', '35-49 ans', '60 ans et +', '25-34 ans', '3-14 ans', '15-24 ans']
Categories (6, object): ['3-14 ans' < '15-24 ans' < '25-34 ans' < '35-49 ans' < '50-59 ans' < '60 ans et +']

### **Calculez le profil par semaine cinema, par âge**
La colonne créée doit s'appeler 'profil_age'.  
N'oubliez pas qu'il faut utiliser la colonne 'Entrées Extrapolées' pour les calculs (voir les exercices 8 et 9 de la partie 02_Exploration).  
Nommez le résultat data_agg_age et assurez-vous que ce soit une DataFrame.
Les variables qui ont servies à l'agrégation ne doivent plus être en index, mais dans une colonne.

In [ ]:
# Résultat attendu 
# sem_cine_inv	age_cat_6       profil_age	
# 2326	        3-14 ans	    0.159081
#               15-24 ans	    0.171070
#               25-34 ans	    0.136125
#               35-49 ans	    0.192822
#               50-59 ans	    0.140421
#               60 ans et +	    0.200480
# 2327	        3-14 ans	    0.188744
#               15-24 ans	    0.192692
#               25-34 ans	    0.133367
#               35-49 ans	    0.191609
#               50-59 ans	    0.129968
#               60 ans et +	    0.163621
# 2328	        3-14 ans	    0.168159
#               15-24 ans	    0.177405
#               25-34 ans	    0.102985
#               35-49 ans	    0.206507
#               50-59 ans	    0.133237
#               60 ans et +	    0.211706
# 2329	        3-14 ans	    0.137017
#               15-24 ans	    0.272937
#               25-34 ans	    0.152800
#               35-49 ans	    0.182990
#               50-59 ans	    0.108096
#               60 ans et +	    0.146159
# 2330	        3-14 ans	    0.134854
#               15-24 ans	    0.238800
#               25-34 ans	    0.166774
#               35-49 ans	    0.184379
#               50-59 ans	    0.112589
#               60 ans et +	    0.162604


data_agg_age = (
    (
        data_weighted.groupby(['sem_cine','age_cat_6'], observed=True, )['Entrées Extrapolées'].sum()
        /data_weighted.groupby(['sem_cine',])['Entrées Extrapolées'].sum()
).to_frame('profil_age')
.reset_index()
)
data_agg_age

,sem_cine,age_cat_6,profil_age
0,2326,3-14 ans,0.159081
1,2326,15-24 ans,0.171070
2,2326,25-34 ans,0.136125
3,2326,35-49 ans,0.192822
4,2326,50-59 ans,0.140421
5,2326,60 ans et +,0.200480
6,2327,3-14 ans,0.188744
7,2327,15-24 ans,0.192692
8,2327,25-34 ans,0.133367
9,2327,35-49 ans,0.191609


### **Créez un graphique en barres des profils par âge en fonction de la semaine cinéma**
Utilisez plotly.express   
Renseignez vous sur les paramètres à utiliser en consultant la doc ici : https://plotly.com/python-api-reference/generated/plotly.express.bar     
Sauvegardez le graphique dans une varibale appelée fig.

In [16]:
fig = px.bar(
    data_agg_age,
    x='sem_cine',
    y='profil_age'
)
fig

### **Changez la couleurs des lignes en utilisant la palette age_colors ci-dessous**  
N'hésitez pas à utiliser le liden de la documentation plotly.  
Lisez le nom des paramètres disponibles. Si le nom d'un paramètres vous interpelle, scrollez vers le bas pour voir son utilisation.

In [18]:
age_colors = [
    f'rgba(153,204,255,1)',
    f'rgba(64,153,255,1)',
    f'rgba(153,51,204,1)',
    f'rgba(102,1,153,1)',
    f'rgba(246,102,103,1)',
    f'rgba(204,51,51,1)',
]

fig = px.bar(
    data_agg_age,
    x='sem_cine',
    y='profil_age',
    color='age_cat_6',
    color_discrete_sequence=age_colors,
)
fig

### **Ajoutez les valeurs formattées de chaque profil**  
Dans le doc au lien donnée ci-dessus, trouver les paramètres qui permettent : 
- d'ajouter la valeur de chaque profil dans chaque barre.  
- de formatter les valeurs en pourcentage avec 1 décimale.  

Vous trouverez ci-dessous les différentes possibilités de formattages possibles:  
| Format  | Description                                               | Example output for `12345.678`         |
| ------- | --------------------------------------------------------- | -------------------------------------- |
| `d`     | Integer (no decimals)                                     | `12346`                                |
| `,d`    | Integer with thousands separator                          | `12,346`                               |
| `.2f`   | Fixed-point, 2 decimals                                   | `12345.68`                             |
| `,.1f`  | Fixed-point with thousands separator, 1 decimal           | `12,345.7`                             |
| `e`     | Exponential notation                                      | `1.234568e+4`                          |
| `.2e`   | Exponential with 2 decimals                               | `1.23e+4`                              |
| `r`     | Rounded significant digits                                | `12300`                                |
| `.3r`   | 3 significant digits                                      | `12300`                                |
| `s`     | SI-prefix notation                                        | `12k`                                  |
| `.2s`   | SI-prefix with 2 significant digits                       | `12k`                                  |
| `%`     | Percentage (value ×100 with `%`)                          | `1234568%`                             |
| `.1%`   | Percentage with 1 decimal                                 | `1234567.8%`                           |
| `$,.2f` | Currency (USD style) with thousands separator, 2 decimals | `$12,345.68`                           |
| `.1f€`  | Custom suffix (Euro)                                      | `12345.7€`                             |
| `+,.2f` | Force sign (+/−) with thousands separator, 2 decimals     | `+12,345.68`                           |
| `(.2f`  | Negative numbers in parentheses                           | `12345.68` (if negative: `(12345.68)`) |  

In [ ]:
fig = px.bar(
    data_agg_age,
    x='sem_cine',
    y='profil_age',
    color='age_cat_6',
    color_discrete_sequence=age_colors,
    text='profil_age',
    text_auto='.1%', #type: ignore => remove the warning of wrong passed type
)

fig

### **Autre méthode**  
Vous avez dû normalement trouver deux paramètres à passer à px.bar.  
Seulement, ces paramètres permettent un formattage limité.  
Il n'est par exemple pas possible de régler la position des valeurs dans la barre, où de les mettre en gras.  
Pour contourner cette limite, il est possible d'utiliser la méthode fig.update_traces(), et de passer des arguments utilisables avec go.Bar  
=> https://plotly.com/python-api-reference/generated/plotly.graph_objects.Bar.html

En utilisant les paramètres suivants :  
- texttemplate
- textposition
- insidetextanchor

Assurez-vous que les valeurs soient centrées verticalement et apparaissent en gras (utilisez la balise html <b></b> pour cela).

In [48]:
fig.update_traces(
    texttemplate="<b>%{y:.1%}</b>",
    textposition="inside",
    insidetextanchor="middle"
)
fig

### **Modifier le layout du graphique avec la méthode fig.update_layout()**  
Lien vers la doc : https://plotly.com/python-api-reference/generated/plotly.graph_objects.Layout.html#plotly.graph_objects.Layout  

Ajouter un titre, modifier la hauteur et la largeur à l'aide des paramètres suivants  
- title = dict(  
    * text=..., # str : texte à ajouter  
    font=dict( # modifie la taille, la police, mise en gras etc.  
        size: ... # int  
        family: ... # str ; change la police 'Arial' etc  
        weight: ... # int ou str : mettre en gras ('bold' en anglais)  
    )  
    x= ... #float entre 0 et 1
    xanchor= ... #str 'left', 'center', 'right', permet d'aligner le titre horizontalement
    y=... #float entre 0 et 1
    yanchor=... #str 'top', 'middle', 'center', permet d'aligner le titre verticalement
)  
- height: ... # int  
- width: ... # int 

Lien vrs la doc du titre : https://plotly.com/python-api-reference/generated/plotly.graph_objects.layout.html#plotly.graph_objects.layout.Title

In [49]:
fig.update_layout(
    title=dict(
        text='Forte augmentation de la part des 15-34 ans à la fin du mois de Juillet', 
        font=dict(
            size=20,
            family='Century Gothic',
            weight='bold',
            ),
        x=0.46,
        xanchor='center',
        ),
    height=600,
    width=1000,
)

### **Utilisez fig.update_layout(xaxis=dict()) pour personnaliser l'axe des abscisses**  
Lien vers la doc pour découvrir toutes les fonctionnalités : https://plotly.com/python/reference/layout/xaxis/

Par défaut, Plotly affiche les valeurs présentes dans la colonne utilisée pour l'axe des x.  
Mais le formattage de correspond pas souvent à ce que l'on souhaite.  
Ici par exemple, on souhaiterait afficher "Sem XX" où XX est le numéro de la semaine.  
  
Quand on souhaite afficher des valeurs complétement différentes des valeurs par défaut, il faut trois paramètres:  
- tickmode = 'array'  
- tickvals = ... # une liste des valeurs présentes dans la colonne utilisée pour l'axe des x  
- ticktext = ... # Une liste des valeurs à afficher  

Il est également possible de personnaliser la taille, la police, etc. avec le paramètre tickfont.  
Enfin vous pouvez changer le titre de l'axe avec le paramètre title=dict().  
Ces deux derniers paramètres s'utilisent comme dans la case précédente.  

In [50]:
fig.update_layout(
    xaxis=dict(
        title=dict(text=None),
        tickmode='array', 
        tickvals=data_agg_age['sem_cine'].unique().tolist(), 
        ticktext=[f"Sem {int(i)}" for i in np.arange(26, 31, 1)],
        tickfont=dict(size=12, color='black', weight='bold'),
        ),
)
fig

### **Utilisez fig.update_layout(yaxis=dict()) pour personnaliser l'axe des ordonnées**  
Lien vers la doc pour découvrir toutes les fonctionnalités : https://plotly.com/python/reference/layout/yaxis/

Ici, ce qu'on retrouver en ordonnées est évident : il n'y a donc pas besoin de titre.  

Par défaut, Plotly affiche les valeurs présentes dans la colonne utilisée pour l'axe des y.  
Mais le formattage de correspond pas souvent à ce que l'on souhaite.  
Ici par exemple, on a les bonnes valeurs, mais on voudrait les afficher %.
  
Ici, on souhaite donc juste modifier le format d'affichage. Plotly utilise le format D3.  
C'est possible de le modifier avec le paramètre suivant :  
- tickformat = ... # D3-format string   

Voici des exemples de format à passer à tickformat  
  
| Format  | Description                                               | Example output for `12345.678`         |
| ------- | --------------------------------------------------------- | -------------------------------------- |
| `d`     | Integer (no decimals)                                     | `12346`                                |
| `,d`    | Integer with thousands separator                          | `12,346`                               |
| `.2f`   | Fixed-point, 2 decimals                                   | `12345.68`                             |
| `,.1f`  | Fixed-point with thousands separator, 1 decimal           | `12,345.7`                             |
| `e`     | Exponential notation                                      | `1.234568e+4`                          |
| `.2e`   | Exponential with 2 decimals                               | `1.23e+4`                              |
| `r`     | Rounded significant digits                                | `12300`                                |
| `.3r`   | 3 significant digits                                      | `12300`                                |
| `s`     | SI-prefix notation                                        | `12k`                                  |
| `.2s`   | SI-prefix with 2 significant digits                       | `12k`                                  |
| `%`     | Percentage (value ×100 with `%`)                          | `1234568%`                             |
| `.1%`   | Percentage with 1 decimal                                 | `1234567.8%`                           |
| `$,.2f` | Currency (USD style) with thousands separator, 2 decimals | `$12,345.68`                           |
| `.1f€`  | Custom suffix (Euro)                                      | `12345.7€`                             |
| `+,.2f` | Force sign (+/−) with thousands separator, 2 decimals     | `+12,345.68`                           |
| `(.2f`  | Negative numbers in parentheses                           | `12345.68` (if negative: `(12345.68)`) |  


Pensez également à bien choisir le nombre de ticks à afficher.  
C'est notamment pratique sur un diagramme en barres empilées pour afficher le tick des 50% !   
Il existe :  
- nticks = ... #int nombre maximum de ticks. Peu pratique.  
- dtick= ... #unité de l'axe. Plus fiable  

In [62]:
fig.update_layout(
    yaxis=dict(
        title=dict(text=None),
        tickformat='.0%',
        dtick=0.25 # Affiche le tick des 50% absent par défaut
    ),
)
fig

### **Personnaliser la légende avec fig.update_layout(legend=dict())** 
La légende est toujours un sujet de discorde : faut-il la laisser ou l'enlever ?  
  
Pour l'enlever, utiliser fig.update_layout(showlegend=False)
  
Si vous la laisser voici quelques paramètres utiles:
fig.update_layout(  
- legend=dict(  
    - title=dict(...) # le titre, par default le nom de la colonne servant à la couleur, la forme etc.  
    - orientation=...# str, 'v' pour vrticale, 'h' pour horizontale  
    - xanchor=... #str, 'left', 'center', 'right'. Permet d'ajuster la position de la légende selon x   
    - x=... # float la coordonnée en x où d'ajuste la légende   
    - yanchor=... #str, 'top', 'middle', 'bottom'. Permet d'ajuster la position de la légende selon y   
    - y=... # float la coordonnée en y où d'ajuste la légende   
    - font=dict(...) # Personnalise la taille, la police, etc. 
    - traceorder=... #str => Choisir 'order dans lequel les items sont affichés 
- )  

)

Il y a plein d'autres paramètres que vous pouvez découvrir ici : https://plotly.com/python/reference/layout/#layout-showlegend

In [63]:
fig.update_layout(
    legend=dict(
        title=dict(text=None),
        orientation='v',
        x=1,
        xanchor='left',
        y=0.95,
        yanchor='top',
        font=dict(size=12, color='black',weight=100),
        traceorder='reversed', # Afficher la légende dans le sens des barres est plus lisible
    ),
)
fig

### **Enlever le background gris et la grille (traits verticaux et horizontaux)**
Plotly permet d'utiliser certains templates avec fig.update_layout(template=...)

Voici quelques exemples :  
| Template name    | Description                                        |
| ---------------- | -------------------------------------------------- |
| `"plotly"`       | Default light theme (white background, gray axes)  |
| `"plotly_white"` | Minimal white background                           |
| `"plotly_dark"`  | Dark background with contrasting gridlines         |
| `"ggplot2"`      | Inspired by R’s ggplot2 style                      |
| `"seaborn"`      | Inspired by Seaborn’s defaults                     |
| `"simple_white"` | Pure white background, no gridlines                |
| `"none"`         | No theming at all (useful for full manual control) |
| ---------------- | -------------------------------------------------- |
| `"presentation"` | Clean style for slides/presentations               |
| `"xgridoff"`     | Like default but gridlines off                     |
| `"ygridoff"`     | Like default but y gridlines off                   |
| `"gridon"`       | Like default but emphasizes grid                   |
| `"none"`         | Removes template (barebones)                       |


Personnellement j'aime beaucoup le template 'plotly_white'. Minimal permet de se concentrer sur l'essentiel.  
Il suffit donc de coder : fig.update_layout(template='plotly_white')  

Il est possible de coder sans le template. Voici ce qu'il faut faire 'plotly_white' :  
fig.update_layout(  
- plot_bgcolor="white",   # background inside the plotting area  
    paper_bgcolor="white",  # background outside the plotting area  
    xaxis=dict(  
    - showgrid=True,      # show gridlines  
        gridcolor="lightgray",  
        zeroline=False      # no bold line at x=0  
        ),  
    yaxis=dict(  
    - showgrid=True,  
        gridcolor="lightgray",  
        zeroline=False  
        )  

)

In [64]:
fig.update_layout(template='plotly_white')
fig

### **Travailler et formatter les hover_data**
L'intérêt de plotly est son interactivité. Lorsque vous passez le curseur de la souris sur chaque barre, des données apparaissent : les hover_data.  
Il s'agit des données que vous avez passez à plotly lors de la création de la figure : 'sem_cine', 'profil_age', 'age_cat_6'.  

Il est tout à fait possible de retirer certaines de ces données, d'en ajouter, de les formatter, etc.
Pour cela, on va s'intéresser fig.update_traces() et à deux paramètres:  
- customdata : il vous permet de passer des données à ajouter aux hover_data avec df[columns].  
- hovertemplate : il vous permet de sélectionner et de formater comme souhaiter les données à afficher. Passez une str.  

Le formattage de hovertemplate répond à des règles précises:  
- Choisir son placeholder, le mettre entre {}, précédé de %  
    * %{x} → la coordonnée en abscisse du point     
    * %{y} → la coordonnée en ordonnée  
    * %{text} → le texte si vous avez passé text=... à px  
    * %{customdata[i]} → l'élément à l'index i dans customdata. <b>Si un seul élement => %{customdata}</b> 
- Choisir le formattage :  
    * %{y:.1f} → valeur de y avec 1 décimale  
    * %{customdata[1]:.2f} → affiche la seconde données de customdata avec 2 décimales  
    * %{x:.1%} → valeur de x affichée en pourcentage avec 1 décimale  
    * %{x:,d} → valeur entière de x avec séparateur de milliers (1,234,567)  
    * <b>%{text}</b>→ text écrit en gras (utilisation de balise html => balise ouvrante b entre <>; balise fermante \b entre <>)  


Formattez les données avec hovertemplate comme suit :  
- sem_cine en gras  
- age_cat_6 en gras 
- profil_25+ en % avec 2 décimales   

Chaque donnée doit être précédée de sa signification business. Revenir à la ligne après chaque donnée (balise br entre <>)

In [83]:
fig.update_traces(
    customdata=data_agg_age['age_cat_6'],
    hovertemplate="<br>".join(
        [   "<b>%{x}</b>",
            "Catégorie d'âge : <b>%{customdata}</b>",
            "Part : <b>%{y:.1%}</b>",
            "<extra></extra>"
        ]
    ),
)
fig

### **Faire le graphique en utilisant les catégories d'habitude de fréquetation suivantes :**
- Assidus (1 & 2)
- Réguliers (3 & 4)
- Occasionnels (5, 6, 7 & 8)

Nommer la colonne 'habfreq_3'  
Assurez-vous que la colonne soit bien de type category au moment où vous la créez.  

Lors du calcul des profils, vous nommerez :  
- la dataframe de résultats : data_agg_habfreq  
- La colonne avec les profils : 'profil_habfreq'  

Nommez la figure fig_habfreq. Utilisez les couleurs ci-dessous.

In [84]:
habfreq_colors = [
    "rgba(0,0,102,1)", #Assidus
    "rgba(57,69,185,1)", #Réguliers
    "rgba(150,157,222,1)", #Occasionnels
]

In [87]:
data_weighted['habfreq_3'] = pd.cut(data_weighted['s3ad'], 
        bins=[0, 2, 4, 8], 
        labels=['Assidus', 'Réguliers', 'Occasionnels',]
    )
data_weighted['habfreq_3'].unique()

['Assidus', 'Réguliers', 'Occasionnels']
Categories (3, object): ['Assidus' < 'Réguliers' < 'Occasionnels']

In [88]:
# Résultat attendu 
#   sem_cine	habfreq_3	    profil_habfreq
#   0	2326	Habitués	    0.165083
#   1	2326	Réguliers	    0.474787
#   2	2326	Occasionnels	0.360130
#   3	2327	Habitués	    0.176426
#   4	2327	Réguliers	    0.445113
#   5	2327	Occasionnels	0.378461
#   6	2328	Habitués	    0.176271
#   7	2328	Réguliers	    0.475855
#   8	2328	Occasionnels	0.347875
#   9	2329	Habitués	    0.175218
# 10	2329	Réguliers	    0.470611
# 11	2329	Occasionnels	0.354171
# 12	2330	Habitués	    0.183535
# 13	2330	Réguliers	    0.452334
# 14	2330	Occasionnels	0.364131

data_agg_habfreq = (
    (
        data_weighted.groupby(['sem_cine','habfreq_3'], observed=True, )['Entrées Extrapolées'].sum()
        /data_weighted.groupby(['sem_cine',])['Entrées Extrapolées'].sum()
).to_frame('profil_habfreq')
.reset_index()
)
data_agg_habfreq

,sem_cine,habfreq_3,profil_habfreq
0,2326,Assidus,0.165083
1,2326,Réguliers,0.474787
2,2326,Occasionnels,0.360130
3,2327,Assidus,0.176426
4,2327,Réguliers,0.445113
5,2327,Occasionnels,0.378461
6,2328,Assidus,0.176271
7,2328,Réguliers,0.475855
8,2328,Occasionnels,0.347875
9,2329,Assidus,0.175218


In [ ]:
fig_habfreq = px.bar(
    data_agg_habfreq,
    x='sem_cine',
    y='profil_habfreq',
    color='habfreq_3',
    color_discrete_sequence=habfreq_colors,
    text='profil_habfreq',
    text_auto='.1%', #type: ignore => remove the warning of wrong passed type
)

fig_habfreq.update_traces(
    texttemplate="<b>%{y:.1%}</b>",
    textposition="inside",
    insidetextanchor="middle",
    customdata=data_agg_habfreq['habfreq_3'],
    hovertemplate="<br>".join(
        [   "<b>%{x}</b>",
            "Catégorie d'habitude de fréquentation : <b>%{customdata}</b>",
            "Part : <b>%{y:.1%}</b>",
            "<extra></extra>"
        ]
    ),
)

fig_habfreq.update_layout(
    height=600,
    width=1000,
    title=dict(
        text='Assiduité stable des spectateurs au cours du mois de Juillet', 
        font=dict(
            size=20,
            family='Century Gothic',
            weight='bold',
            ),
        x=0.46,
        xanchor='center',
        ),
    xaxis=dict(
        title=dict(text=None),
        tickmode='array', 
        tickvals=data_agg_habfreq['sem_cine'].unique().tolist(), 
        ticktext=[f"Sem {int(i)}" for i in np.arange(26, 31, 1)],
        tickfont=dict(size=12, color='black', weight='bold'),
        ),
    yaxis=dict(
        title=dict(text=None),
        tickformat='.0%',
        dtick=0.25 # Affiche le tick des 50% absent par défaut
    ),
    legend=dict(
        title=dict(text=None),
        orientation='v',
        x=1,
        xanchor='left',
        y=0.95,
        yanchor='top',
        font=dict(size=12, color='black',weight=100),
        traceorder='reversed', # Afficher la légende dans le sens des barres est plus lisible
    ),
    template='plotly_white'
)

fig_habfreq

### **Faire le graphique en utilisant les catégories de sexe suivantes :**
- Hommes (1)
- Femmes (2)

Nommer la colonne 'gender_2'  
Assurez-vous que la colonne soit bien de type category au moment où vous la créez.  

Lors du calcul des profils, vous nommerez :  
- la dataframe de résultats : data_agg_gender  
- La colonne avec les profils : 'profil_gender'  

Nommez la figure fig_gender. Utilisez les couleurs ci-dessous.

In [97]:
gender_colors = [
    "rgba(191,216,238,1)", #Hommes
    "rgba(0,98,188,1)", #Femmes
]

In [90]:
data_weighted['gender_2'] = pd.cut(data_weighted['gender'], 
        bins=[0, 1, 2], 
        labels=['Hommes', 'Femmes', ]
    )
data_weighted['gender_2'].unique()

['Hommes', 'Femmes']
Categories (2, object): ['Hommes' < 'Femmes']

In [ ]:
# Résultat attendu 
#   sem_cine	gender_2	profil_gender
# 0	2326	    Hommes	    0.531544
# 1	2326	    Femmes	    0.468456
# 2	2327	    Hommes	    0.496985
# 3	2327	    Femmes	    0.503015
# 4	2328	    Hommes	    0.514009
# 5	2328	    Femmes	    0.485991
# 6	2329	    Hommes	    0.489497
# 7	2329	    Femmes	    0.510503
# 8	2330	    Hommes	    0.524315
# 9	2330	    Femmes	    0.475685

data_agg_gender = (
    (
        data_weighted.groupby(['sem_cine','gender_2'], observed=True, )['Entrées Extrapolées'].sum()
        /data_weighted.groupby(['sem_cine',])['Entrées Extrapolées'].sum()
).to_frame('profil_gender')
.reset_index()
)
data_agg_gender

,sem_cine,gender_2,profil_gender
0,2326,Hommes,0.531544
1,2326,Femmes,0.468456
2,2327,Hommes,0.496985
3,2327,Femmes,0.503015
4,2328,Hommes,0.514009
5,2328,Femmes,0.485991
6,2329,Hommes,0.489497
7,2329,Femmes,0.510503
8,2330,Hommes,0.524315
9,2330,Femmes,0.475685


In [104]:
texte_homme = f"<span style='color:{gender_colors[0]}'>tantôt masculin</span>"
texte_femme = f"<span style='color:{gender_colors[1]}'>tantôt féminin</span>"

fig_gender = px.bar(
    data_agg_gender,
    x='sem_cine',
    y='profil_gender',
    color='gender_2',
    color_discrete_sequence=gender_colors,
    text='profil_gender',
    text_auto='.1%', #type: ignore => remove the warning of wrong passed type
)

fig_gender.update_traces(
    texttemplate="<b>%{y:.1%}</b>",
    textposition="inside",
    insidetextanchor="middle",
    customdata=data_agg_gender['profil_gender'],
    hovertemplate="<br>".join(
        [   "<b>%{x}</b>",
            "Sexe : <b>%{customdata}</b>",
            "Part : <b>%{y:.1%}</b>",
            "<extra></extra>"
        ]
    ),
)

fig_gender.update_layout(
    height=600,
    width=1000,
    title=dict(
        #Il est possible de customisrer le titre en matrisant un peu le langage html
        # => "Un public hebdomadaire <span style='color:rgba(191,216,238,1)'>tantôt masculin</span>, <span style='color:rgba(0,98,188,1)'>tantôt féminin</span>"
        # la phrase est peu lisible et non dynamique
        # => f"Un public hebdomadaire <span style='color:{gender_colors[0]}'>tantôt masculin</span>, <span style='color:{gender_colors[1]}'>tantôt féminin</span>"
        # phrase dynamique mais peu lisible
        # Faire en 3 fois : 
        # texte_homme = f"<span style='color:{gender_colors[0]}'>tantôt masculin</span>"
        # texte_femme = f"<span style='color:{gender_colors[1]}'>tantôt féminin</span>"
        # f"Un public hebdomadaire {texte_homme}, {texte_femme}"
        text=f"Un public hebdomadaire {texte_homme}, {texte_femme}", 
        font=dict(
            size=20,
            family='Century Gothic',
            weight='bold',
            ),
        x=0.46,
        xanchor='center',
        ),
    xaxis=dict(
        title=dict(text=None),
        tickmode='array', 
        tickvals=data_agg_gender['sem_cine'].unique().tolist(), 
        ticktext=[f"Sem {int(i)}" for i in np.arange(26, 31, 1)],
        tickfont=dict(size=12, color='black', weight='bold'),
        ),
    yaxis=dict(
        title=dict(text=None),
        tickformat='.0%',
        dtick=0.25 # Affiche le tick des 50% absent par défaut
    ),
    legend=dict(
        title=dict(text=None),
        orientation='v',
        x=1,
        xanchor='left',
        y=0.95,
        yanchor='top',
        font=dict(size=12, color='black',weight=100),
        traceorder='reversed', # Afficher la légende dans le sens des barres est plus lisible
    ),
    template='plotly_white'
)

fig_gender